In [169]:
from itertools import chain, combinations

In [170]:
class Graph:
    def __init__(self,edges):
        self.edges = edges
        self.nodes = sorted({ f for f,t in edges }.union({ t for f,t in edges }))
        
        # Perform a DFS from the first node and reorder nodes based on DFS
        

In [171]:
# Graph from the DPccp paper

g1 = Graph({("R0","R1"),
            ("R0","R2"),
            ("R0","R3"),
            ("R1","R4"),
            ("R2","R3"),
            ("R2","R4"),
            ("R3","R4")})

In [172]:
g2 = Graph({("R0","R1"),
            ("R1","R2"),
            ("R2","R3"),
            ("R3","R4"),
            ("R0","R4")})

In [173]:
g3 = Graph({("R0","R1"),
            ("R1","R2"),
            ("R2","R3"),
            ("R0","R3")})

In [174]:
dptable = {}

In [175]:
def makeBi(G,n):
    return {x for x in G.nodes if x <= n}

In [176]:
def Neighborhood(G,S,X={}):
    res = []
    for s in S:
        res += [ x for x,y in G.edges if y==s]
        res += [ y for x,y in G.edges if x==s]
    return set(res).difference(X)

In [177]:
# Enumerate subsets with subsets first
def enumerateSubsets(S):
    sS = sorted(S)
    "powerset([1,2,3]) → (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    return chain.from_iterable(combinations(sS, r) for r in range(1,len(sS)+1))

In [178]:
def enumerateCsg(G):
    for n in reversed(G.nodes):
        yield {n}
        for x in enumerateCsgRec(G,{n},makeBi(G,n)):
            yield x

In [179]:
def enumerateCsgRec(G,S,X,tab=0):
    print("\t"*tab + "calling enumerateCsgRec with S=",S, ", X=", X)
    N = Neighborhood(G,S).difference(X)
    for S1 in enumerateSubsets(N):
        yield S.union(set(S1))
    for S1 in enumerateSubsets(N):
        for x in enumerateCsgRec(G,S.union(set(S1)),X.union(N),tab+1):
            yield x

In [180]:
def makeB(G,N,i):
    return {x for x in N if x <= i }

In [181]:
def enumerateCmp(G,S):
    print("calling enumerateCmp with S=",S)
    X = makeBi(G,min(S)).union(S)
    N = Neighborhood(G,S).difference(X)
    for n in reversed(sorted(N)):
        yield {n}
        for x in enumerateCsgRec(G,{n},X.union(makeB(G,N,n))):
            yield x

In [187]:
class solver:
    def __init__(self,G):
        self.G = G
        self.dp = {}
        self.check_table ={}
        self.errors = 0
        self.counter = 0
    
    def solve(self):
        for n in reversed(self.G.nodes):
            self.dp[frozenset({n})] = True
            
        for n in reversed(self.G.nodes):
            self.emitCsg({n})
            self.enumerateCsgRec({n},makeBi(self.G,n))
            
        print("Pairs # ",len(self.check_table))
        print("Counter =",self.counter)
        print("Duplicates: ", self.errors)
        print("Extra checks = ", self.errors/len(self.check_table))
            
    def emitCsg(self,S,tab=0):
        print("\t"*tab + "Emit csg, S=",sorted(S))
        X = makeBi(self.G,min(S)).union(S)
        N = Neighborhood(self.G,S,X)
        for n in reversed(sorted(N)):
            self.emitCsgCmp(S,{n},tab+1)
            self.enumerateCmpRec(S,{n},X.union(makeB(self.G,N,n)),tab+1)
        
    def enumerateCsgRec(self,S1,X,tab=0):
        print("\t"*tab + "Csg rec, S1=",sorted(S1),", X=",sorted(X))
        NS = Neighborhood(self.G,S1,X)
        for N in enumerateSubsets(NS):
            self.emitCsg(S1.union(set(N)),tab+1)
        for N in enumerateSubsets(NS):
            self.enumerateCsgRec(S1.union(set(N)),X.union(NS),tab+1)
                         
    def enumerateCmpRec(self,S1,S2,X,tab=0):
        print("\t"*tab + "Cmp rec, S1=",sorted(S1),", S2=", sorted(S2),", X=",sorted(X))
        
        NS = Neighborhood(self.G,S2,X)
        for N in enumerateSubsets(NS):
            if frozenset(S2.union(set(N))) in self.dp:
                self.emitCsgCmp(S1,S2.union(set(N)),tab+1)
        #X = X.union(NS)
        #NS = Neighborhood(self.G,S2,X)
        for N in enumerateSubsets(NS):
            self.enumerateCmpRec(S1,S2.union(N),X.union(NS),tab+1)
            
    def emitCsgCmp(self,S1,S2,tab=0):
        # check that S1 and S2 are in dpTable
        if not frozenset(S1) in self.dp:
            print ("ERROR: ",S1, " is not in dpTable")
            
        if not frozenset(S2) in self.dp:
            print ("ERROR: ",S2, " is not in dpTable")
            
        S1S2 = S1.union(S2)
        self.dp[frozenset(S1S2)] = True
        
        print("\t"*tab + "Added PAIR: ", sorted(S1), " and ", sorted(S2))
        pair = (frozenset(S1),frozenset(S2))
        if pair in self.check_table:
            print("ERROR: pair already processed")
            self.errors += 1
            
        self.check_table[pair] = True
        self.counter += 1

In [190]:
s = solver(g1)
s.solve()

Emit csg, S= ['R4']
Csg rec, S1= ['R4'] , X= ['R0', 'R1', 'R2', 'R3', 'R4']
Emit csg, S= ['R3']
	Added PAIR:  ['R3']  and  ['R4']
	Cmp rec, S1= ['R3'] , S2= ['R4'] , X= ['R0', 'R1', 'R2', 'R3', 'R4']
Csg rec, S1= ['R3'] , X= ['R0', 'R1', 'R2', 'R3']
	Emit csg, S= ['R3', 'R4']
	Csg rec, S1= ['R3', 'R4'] , X= ['R0', 'R1', 'R2', 'R3', 'R4']
Emit csg, S= ['R2']
	Added PAIR:  ['R2']  and  ['R4']
	Cmp rec, S1= ['R2'] , S2= ['R4'] , X= ['R0', 'R1', 'R2', 'R3', 'R4']
	Added PAIR:  ['R2']  and  ['R3']
	Cmp rec, S1= ['R2'] , S2= ['R3'] , X= ['R0', 'R1', 'R2', 'R3']
		Added PAIR:  ['R2']  and  ['R3', 'R4']
		Cmp rec, S1= ['R2'] , S2= ['R3', 'R4'] , X= ['R0', 'R1', 'R2', 'R3', 'R4']
Csg rec, S1= ['R2'] , X= ['R0', 'R1', 'R2']
	Emit csg, S= ['R2', 'R3']
		Added PAIR:  ['R2', 'R3']  and  ['R4']
		Cmp rec, S1= ['R2', 'R3'] , S2= ['R4'] , X= ['R0', 'R1', 'R2', 'R3', 'R4']
	Emit csg, S= ['R2', 'R4']
		Added PAIR:  ['R2', 'R4']  and  ['R3']
		Cmp rec, S1= ['R2', 'R4'] , S2= ['R3'] , X= ['R0', 'R1', 'R2'

In [184]:
def opt(G):
    # Create DP table:
    dpTable = {}
    checkTable = {}
    counter = 0
    
    # Create duplicate pair table for correctness testing
    dupTable = {}
    
    # iterate over enumeration of connected subgraphs:
    for x in enumerateCsg(G):
        # insert the subgraph into dptable if its a singleton
        fx = frozenset(x)
        if len(x)==1:
            dpTable[fx] = True
            #print ("Added CSG: ",x)
        
        # Now compute all the complements of the subset
        for y in enumerateCmp(G,x):
            # check that the compliment is already in the dpTable
            fy = frozenset(y)
            if fy not in dpTable:
                print("ERROR (cmp): key ", sorted(y), " is not in the table, but should be")

            print ("Added PAIR: ", sorted(x), " and ", sorted(y))
            if frozenset({fx,fy}) in dupTable:
                print("ERROR: pair (", sorted(x), ",", sorted(y), ") is in the table, but should not be")
                
            dupTable[ frozenset({fx,fy}) ] = True


            # add the combintation of csg and cmp to the dpTable
            combo = x.union(y)
            comboKey = frozenset(combo)
            dpTable[comboKey] = True
            counter += 1
            
            pair = (fx,fy)
            checkTable[pair] = True
            
    print ("Total pairs considered:",counter)
    print ("Check table size:",len(checkTable))
    return checkTable

In [185]:
t2 = opt(g1)

calling enumerateCmp with S= {'R4'}
calling enumerateCsgRec with S= {'R4'} , X= {'R0', 'R1', 'R4', 'R2', 'R3'}
calling enumerateCmp with S= {'R3'}
Added PAIR:  ['R3']  and  ['R4']
calling enumerateCsgRec with S= {'R4'} , X= {'R0', 'R4', 'R1', 'R2', 'R3'}
calling enumerateCsgRec with S= {'R3'} , X= {'R0', 'R1', 'R2', 'R3'}
calling enumerateCmp with S= {'R4', 'R3'}
	calling enumerateCsgRec with S= {'R4', 'R3'} , X= {'R0', 'R4', 'R1', 'R2', 'R3'}
calling enumerateCmp with S= {'R2'}
Added PAIR:  ['R2']  and  ['R4']
calling enumerateCsgRec with S= {'R4'} , X= {'R0', 'R4', 'R1', 'R2', 'R3'}
Added PAIR:  ['R2']  and  ['R3']
calling enumerateCsgRec with S= {'R3'} , X= {'R0', 'R1', 'R2', 'R3'}
Added PAIR:  ['R2']  and  ['R3', 'R4']
	calling enumerateCsgRec with S= {'R4', 'R3'} , X= {'R0', 'R4', 'R1', 'R2', 'R3'}
calling enumerateCsgRec with S= {'R2'} , X= {'R0', 'R1', 'R2'}
calling enumerateCmp with S= {'R2', 'R3'}
Added PAIR:  ['R2', 'R3']  and  ['R4']
calling enumerateCsgRec with S= {'R4'} , 

In [186]:
for k in t2:
    if not k in s.check_table:
        print (k)